In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
import pdb
import _pickle as cPickle
import time
import subprocess
import glob

import random
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict, Counter
from ncrp import Topic, Doc, init, sample, get_perplexity, get_topic_specialization, get_hierarchical_affinities, get_freq_tokens_ncrp, get_docs
from configure import get_config

# load config & data 

In [3]:
config = get_config(nb_name)
np.random.seed(config.seed)
random.seed(config.seed)

In [4]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

Exception ignored in: <_io.FileIO name='data/20news/instances.pkl' mode='rb' closefd=True>


In [5]:
config.n_doc = len(instances_train)
config.n_vocab = len(bow_idxs)
config.n_doc, config.n_vocab

(11258, 1995)

# run

## initialize log

In [6]:
checkpoint = []
ppl_min = np.inf
epoch = 0

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','VALID:','TEST:','SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','PPL','PPL', 'PPL','1', '2', '3', 'CHILD', 'OTHER']]))))

def update_checkpoint(config, checkpoint, epoch):
    checkpoint.append(config.path_model + '-%i' % epoch)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0)
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize data

In [7]:
topic_root = Topic(idx='0', sibling_idx=0, parent=None, depth=0, config=config)
train_docs = get_docs(instances_train, config)
dev_docs = get_docs(instances_dev, config)
test_docs = get_docs(instances_test, config)
init(train_docs, dev_docs, test_docs, topic_root)

0 10000 0 0 

## run

In [ ]:
time_start = time.time()
while epoch < 100:
# while epoch < config.n_epochs:
    sample(train_docs, dev_docs, test_docs, topic_root)
    ppl_train = get_perplexity(train_docs, topic_root)
    ppl_dev = get_perplexity(dev_docs, topic_root)
    
    if ppl_dev < ppl_min:
        ppl_min = ppl_dev
        ppl_test = get_perplexity(test_docs, topic_root)
        cPickle.dump([test_docs, topic_root], open(config.path_model + '-%i'%epoch, 'wb'))
        update_checkpoint(config, checkpoint, epoch)
        
    depth_spec = get_topic_specialization(test_docs, topic_root)
    hierarchical_affinities = get_hierarchical_affinities(topic_root)
    
    clear_output()
    time_log = int(time.time() - time_start)
    time_start = time.time()
    log_series = pd.Series([time_log, epoch, 0, \
            '%.0f'%ppl_train, '%.0f'%ppl_dev, '%.0f'%ppl_test, \
            '%.2f'%depth_spec[1], '%.2f'%depth_spec[2], '%.2f'%depth_spec[3], \
            '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
            index=log_df.columns)
    log_df.loc[epoch] = log_series    
    display(log_df)
    get_freq_tokens_ncrp(topic_root, idx_to_word, bow_idxs)
    
    cPickle.dump(log_df, open(config.path_log, 'wb'))
    epoch += 1

TRAIN: VALID: TEST: SPEC:             HIER:      
   Time  Ep Ct    PPL    PPL   PPL     1     2     3 CHILD OTHER
0   474   0  0   1031   1030  1035  0.01  0.44  0.44  0.70  0.68
1   546   1  0    957    963   964  0.01  0.46  0.45  0.72  0.71
2   657   2  0    911    927   927  0.01  0.47  0.46  0.76  0.75
3   612   3  0    887    908   908  0.02  0.47  0.46  0.75  0.74
4   599   4  0    867    889   889  0.02  0.47  0.46  0.71  0.70
5   649   5  0    852    872   873  0.03  0.47  0.46  0.73  0.73
6   694   6  0    834    856   857  0.03  0.47  0.46  0.76  0.75
7   674   7  0    818    838   838  0.04  0.46  0.46  0.74  0.73
8   668   8  0    807    826   826  0.04  0.45  0.46  0.74  0.73
9   652   9  0    799    817   817  0.04  0.45  0.46  0.73  0.73
10  663  10  0    795    814   813  0.05  0.45  0.46  0.73  0.73
11  619  11  0    791    810   810  0.05  0.45  0.46  0.67  0.67
12  613  12  0    789    807   807  0.05  0.44  0.46  0.67  0.67
13  623  13  0    787    805   805  0.05  0.44  0.46  0.67  0.66
14  634  14  0    786    804   804  0.05  0.44  0.46  0.70  0.70
15  640  15  0    785    803   802  0.06  0.44  0.46  0.70  0.70
16  635  16  0    784    802   802  0.06  0.44  0.46  0.70  0.69
17  654  17  0    783    801   800  0.06  0.44  0.46  0.70  0.69
18  633  18  0    783    800   800  0.06  0.43  0.46  0.70  0.69
19  640  19  0    783    800   800  0.06  0.43  0.46  0.70  0.69
20  644  20  0    783    800   800  0.06  0.43  0.46  0.70  0.69
21  643  21  0    783    799   799  0.06  0.43  0.46  0.70  0.69
22  633  22  0    782    799   799  0.06  0.43  0.46  0.70  0.69
23  639  23  0    782    798   798  0.06  0.43  0.46  0.70  0.69
24  637  24  0    781    797   798  0.06  0.43  0.46  0.70  0.68
25  641  25  0    781    797   797  0.06  0.43  0.46  0.69  0.68
26  655  26  0    779    796   796  0.06  0.43  0.46  0.69  0.68
27  658  27  0    779    795   795  0.06  0.43  0.46  0.69  0.68
28  656  28  0    778    794   794  0.06  0.43  0.46  0.69  0.68
29  657  29  0    777    794   793  0.06  0.43  0.46  0.69  0.68
30  659  30  0    777    792   793  0.06  0.43  0.46  0.69  0.68
31  657  31  0    776    792   792  0.06  0.43  0.46  0.69  0.68
32  670  32  0    775    791   791  0.06  0.43  0.46  0.69  0.68
33  669  33  0    774    790   790  0.06  0.43  0.46  0.69  0.68
34  674  34  0    774    790   789  0.06  0.43  0.46  0.69  0.67
35  671  35  0    773    789   789  0.06  0.43  0.46  0.69  0.67
36  644  36  0    773    789   788  0.06  0.43  0.46  0.65  0.64
37  645  37  0    772    789   788  0.06  0.43  0.46  0.65  0.64
38  643  38  0    771    788   788  0.06  0.43  0.46  0.65  0.64
39  644  39  0    770    787   787  0.06  0.43  0.46  0.65  0.64
40  640  40  0    770    786   787  0.06  0.43  0.46  0.65  0.64
41  643  41  0    770    786   786  0.06  0.43  0.46  0.65  0.64
42  641  42  0    768    785   785  0.06  0.43  0.46  0.65  0.64
43  653  43  0    768    785   785  0.06  0.43  0.46  0.65  0.63
44  663  44  0    766    783   783  0.06  0.43  0.46  0.65  0.63
45  662  45  0    765    783   783  0.06  0.43  0.46  0.64  0.63
46  652  46  0    765    782   782  0.06  0.43  0.46  0.64  0.63
47  658  47  0    764    781   781  0.06  0.43  0.46  0.64  0.63
48  657  48  0    764    781   780  0.06  0.43  0.46  0.64  0.63
49  646  49  0    763    779   779  0.06  0.43  0.46  0.64  0.63
50  640  50  0    762    778   778  0.06  0.44  0.46  0.64  0.63
51  669  51  0    762    778   778  0.06  0.44  0.46  0.64  0.63
52  662  52  0    761    777   777  0.06  0.44  0.46  0.64  0.63
53  666  53  0    761    777   777  0.06  0.44  0.46  0.64  0.63
54  660  54  0    760    776   775  0.06  0.44  0.46  0.64  0.62
55  658  55  0    759    774   774  0.06  0.44  0.46  0.63  0.62
56  656  56  0    758    773   772  0.06  0.44  0.46  0.63  0.62

Exception ignored in: <_io.FileIO name='model/20news/ncrp/20news-mncrp-alp10.50.1-eta10-gam0.01-epoch50/log' mode='wb' closefd=True>


 0 11258 390258.0 write one article say know think get like make people
   0-1 3667 172318.0 people say go government one gun state make get write
     0-1-1 2128 13492.0 car drive insurance road bike driver ride cop buy safety
     0-1-2 57 13.0 douglas handle meaning either meeting conversation adams posting conference file
     0-1-3 781 2936.0 sexual homosexual man male sex percent gay slave number san
     0-1-4 136 165.0 pope line la de text fool lose genocide mass weapon
     0-1-5 205 249.0 oo smith lose surprise special separate pitch surely treat future
     0-1-6 247 513.0 ted frank document mon editor id apr university privacy police
     0-1-7 113 210.0 hint clipper algorithm clinton idea key blood info regard strongly
   0-2 4203 192649.0 use file get drive program window write system work run
     0-2-1 3798 6809.0 car engine sell bike model battery sale doug condition amp
     0-2-2 220 156.0 nt paper effort family windows goal vote small operate prior
     0-2-3 145 15